In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!ls "/content/drive/MyDrive/"

In [ ]:
cd /content/drive/MyDrive/COS526-Project-main2

In [ ]:
project_path = "/content/drive/MyDrive/COS526-Project-main"


In [ ]:
ls

In [ ]:
ls output_smoke/


In [ ]:
!pip install nuscenes-devkit

In [ ]:
!pip uninstall -y torch torchvision torchaudio xformers

In [ ]:
!pip install --no-cache-dir \
    torch==2.2.1+cu118 \
    torchvision==0.17.1+cu118 \
    torchaudio==2.2.1+cu118 \
  -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
pip install --no-deps accelerate

In [ ]:
!pip install xformers==0.0.25.post1

In [ ]:
!pip install --no-deps accelerate

In [ ]:
!pip install open3d diffusers rembg hy3dgen transformers

In [ ]:
pip install open3d diffusers rembg hy3dgen transformers

In [ ]:
# 1) Force NumPy < 2
!pip install --upgrade "numpy<2"


In [ ]:
# Cell 2: fix torch/torchvision
!pip uninstall -y torch torchvision torchaudio
!pip install --no-cache-dir \
    torch==2.2.1+cu118 \
    torchvision==0.17.1+cu118 \
    torchaudio==2.2.1+cu118 \
    -f https://download.pytorch.org/whl/torch_stable.html
!pip install open3d diffusers rembg hy3dgen xformers accelerate transformers


In [ ]:
!pip uninstall -y torch torchvision torchaudio


In [ ]:
!pip install --upgrade --force-reinstall "numpy<2"
!pip install --no-cache-dir \
    torch==2.2.1+cu118 \
    torchvision==0.17.1+cu118 \
    torchaudio==2.2.1+cu118 \
    -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!pip install -q open3d diffusers rembg hy3dgen xformers accelerate transformers


In [ ]:
import torch, torchvision
from torchvision.ops import nms

print("torch:", torch.__version__, "| cuda:", torch.cuda.is_available())
print("torchvision:", torchvision.__version__)
print("NMS op loaded:", nms)


In [ ]:
pip uninstall -y torch torchvision torchaudio

In [ ]:
pip install --upgrade --force-reinstall "numpy<2"

In [ ]:
!pip install --no-cache-dir \
    torch==2.2.1+cu118 \
    torchvision==0.17.1+cu118 \
    torchaudio==2.2.1+cu118 \
    -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch, torchvision
from torchvision.ops import nms

print("torch:", torch.__version__, "| cuda:", torch.cuda.is_available())
print("torchvision:", torchvision.__version__)
print("NMS op loaded:", nms)


In [ ]:
!pip install -q open3d diffusers rembg hy3dgen xformers accelerate transformers


In [ ]:
import open3d, diffusers, rembg, hy3dgen, xformers, transformers
print("All core libs imported OK")


In [ ]:
# lightweight smoke-test wrapper for Colab
import os, random
import open3d as o3d
from shape_completion_pipelinenuscenes import run_completion

PARTIAL_DIR = "nuscenes_data_output/point_clouds"
GT_DIR      = "nuscenes_data_output/GT"
MIN_POINTS  = 100
NUM_SAMPLES = 2
OUT_DIR     = "output_smoke"
os.makedirs(OUT_DIR, exist_ok=True)

# gather & filter
pc    = {f[:-4] for f in os.listdir(PARTIAL_DIR) if f.endswith(".ply")}
gt    = {f[:-4] for f in os.listdir(GT_DIR)      if f.endswith(".ply")}
good  = [tok for tok in (pc & gt)
         if len(o3d.io.read_point_cloud(f"{PARTIAL_DIR}/{tok}.ply").points) >= MIN_POINTS]

print(f"Found {len(good)} good tokens, sampling {NUM_SAMPLES}…")
sample = random.sample(good, min(NUM_SAMPLES, len(good)))

# run with minimal settings
for tok in sample:
    cd = run_completion(
        token           = tok,
        output_dir      = OUT_DIR,
        gen_rgb         = False,  # skip ControlNet→RGB
        inpaint_depth   = False,  # skip hole-fill
        inference_steps = 10,     # fewer steps
        seed            = 0
    )
    print(f"{tok} → Chamfer {cd:.3f} mm")


In [ ]:
import os, glob
import open3d as o3d

# 1) List exactly what .ply’s are in your folder:
plys = glob.glob("output_smoke/*_mesh.ply")
print("Found these mesh files:", plys)

# 2) Pick one valid path
path = plys[0]
print("Loading mesh from:", path)

# 3) Read it and verify it's non-empty
mesh = o3d.io.read_triangle_mesh(path)
if mesh.is_empty() or len(mesh.triangles) == 0:
    raise RuntimeError(f"Loaded mesh is empty or has no triangles: {path}")

# 4) Sample points and plot via Matplotlib
pcd = mesh.sample_points_uniformly(number_of_points=5000)
pts = np.asarray(pcd.points)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=0.5, c=pts[:,2], cmap='viridis')
ax.view_init(elev=30, azim=45)
ax.set_axis_off()
plt.show()


In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# pick your token
token = "25bfb10d641f42d78edd97c1f9849a66"
pcd_path = f"nuscenes_data_output/point_clouds/{token}.ply"

# read & down-sample for speed
pcd = o3d.io.read_point_cloud(pcd_path)
pcd = pcd.voxel_down_sample(voxel_size=0.01)

# if your runtime can pop up an Open3D window:
o3d.visualization.draw_geometries([pcd])

# otherwise fall back to matplotlib:
pts = np.asarray(pcd.points)
fig = plt.figure(figsize=(6,6))
ax  = fig.add_subplot(111, projection='3d')
ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=1, c=pts[:,2], cmap='Spectral')
ax.set_title(f"Original partial cloud for token {token}")
plt.show()


In [ ]:
import os, glob
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# Find all the completed‐mesh files
mesh_files = glob.glob("output_smoke/*_mesh.ply")
print("Found mesh files:", mesh_files)

for mesh_path in mesh_files:
    # 1) Extract the bare token
    fn = os.path.basename(mesh_path)
    token = fn.replace("_mesh.ply", "")
    print("\nToken:", token)

    # 2) Build path to the **partial** (input) cloud
    partial_pcd_path = f"nuscenes_data_output/point_clouds/{token}.ply"
    if not os.path.exists(partial_pcd_path):
        print("  ✗ no partial cloud at", partial_pcd_path)
        continue

    # 3) Load & down‐sample for speed
    pcd = o3d.io.read_point_cloud(partial_pcd_path)
    print("  Original has", len(pcd.points), "points")
    pcd = pcd.voxel_down_sample(voxel_size=0.02)

    # 4) Plot with matplotlib if Open3D window isn’t available
    pts = np.asarray(pcd.points)
    fig = plt.figure(figsize=(5,5))
    ax  = fig.add_subplot(111, projection='3d')
    ax.scatter(pts[:,0], pts[:,1], pts[:,2],
               c=pts[:,2], cmap="Spectral", s=1)
    ax.set_title(f"Partial cloud for {token}")
    ax.set_axis_off()
    plt.show()


In [ ]:
import os, glob
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# 1) List exactly what .ply’s are in your folder:
plys = glob.glob("output_smoke/*_mesh.ply")
print("Found these mesh files:", plys)

# 2) Pick one valid path
path = plys[1]
print("Loading mesh from:", path)

# 3) Read it and verify it's non-empty
mesh = o3d.io.read_triangle_mesh(path)
if mesh.is_empty() or len(mesh.triangles) == 0:
    raise RuntimeError(f"Loaded mesh is empty or has no triangles: {path}")

# 4) Sample points and plot via Matplotlib
pcd = mesh.sample_points_uniformly(number_of_points=5000)
pts = np.asarray(pcd.points)

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=0.5, c=pts[:,2], cmap='viridis')
ax.view_init(elev=30, azim=45)
ax.set_axis_off()
plt.show()


In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# pick your token
token = "fc510710c89344f2ae4e886f0d2e4c30"
pcd_path = f"nuscenes_data_output/point_clouds/{token}.ply"

# read & down-sample for speed
pcd = o3d.io.read_point_cloud(pcd_path)
pcd = pcd.voxel_down_sample(voxel_size=0.01)

# if your runtime can pop up an Open3D window:
o3d.visualization.draw_geometries([pcd])

# otherwise fall back to matplotlib:
pts = np.asarray(pcd.points)
fig = plt.figure(figsize=(6,6))
ax  = fig.add_subplot(111, projection='3d')
ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=1, c=pts[:,2], cmap='Spectral')
ax.set_title(f"Original partial cloud for token {token}")
plt.show()


In [ ]:
# 1) Downgrade numpy so your wheels will load
!pip install numpy<2

# 2) Install a matching diffusers + hub combo
!pip install diffusers==0.33.1 huggingface_hub==0.14.1

# 3) Then the rest
!pip install open3d rembg hy3dgen xformers transformers accelerate


In [ ]:
!pip install "numpy<2"


In [ ]:
import torch, torchvision
from torchvision.ops import nms
import diffusers, transformers, huggingface_hub, open3d, rembg, hy3dgen, xformers, accelerate

print("torch:           ", torch.__version__, "| cuda:", torch.cuda.is_available())
print("torchvision:     ", torchvision.__version__)
print("diffusers:       ", diffusers.__version__)
print("transformers:    ", transformers.__version__)
print("huggingface_hub: ", huggingface_hub.__version__)
print("NMS op loaded:   ", nms)
print("All core libs imported OK")


In [ ]:
!pip install --upgrade transformers


In [ ]:
import os, random
import numpy as np
import open3d as o3d
from shape_completion_pipelinenuscenes import run_completion

PARTIAL_DIR = "nuscenes_data_output/point_clouds"
GT_DIR      = "nuscenes_data_output/GT"
MIN_POINTS  = 100
MAX_DIST    = 5.0     # meters from origin
NUM_SAMPLES = 5
OUT_DIR     = "output_smoke"
os.makedirs(OUT_DIR, exist_ok=True)

# 1) find tokens present in both dirs
pc_files = {f[:-4] for f in os.listdir(PARTIAL_DIR) if f.endswith(".ply")}
gt_files = {f[:-4] for f in os.listdir(GT_DIR)      if f.endswith(".ply")}
common   = pc_files & gt_files

# 2) filter by point count AND centroid distance
good_tokens = []
for tok in common:
    p = o3d.io.read_point_cloud(f"{PARTIAL_DIR}/{tok}.ply")
    if len(p.points) < MIN_POINTS:
        continue
    centroid = np.asarray(p.get_center())
    dist = np.linalg.norm(centroid)        # distance to (0,0,0)
    if dist <= MAX_DIST:
        good_tokens.append((tok, dist))

# 3) optionally sort by ascending distance
good_tokens.sort(key=lambda x: x[1])

print(f"Found {len(good_tokens)} tokens within {MAX_DIST} m, picking {NUM_SAMPLES}")
# grab just the token names
sample = [tok for tok,_ in good_tokens[:NUM_SAMPLES]]

# 4) run your smoke‐test on the closest ones
for tok in sample:
    cd = run_completion(
        token           = tok,
        output_dir      = OUT_DIR,
        gen_rgb         = False,
        inpaint_depth   = False,
        inference_steps = 10,
        seed            = 0,
    )
    print(f"{tok} (dist {dist:.2f} m) → Chamfer {cd:.3f} mm")


In [ ]:
# ─── lightweight smoke-test wrapper (closest-by-distance) ─────────────────
import os
import numpy as np
import open3d as o3d
from shape_completion_pipelinenuscenes import run_completion

# ─── Config ────────────────────────────────────────────────────────────────
PARTIAL_DIR = "nuscenes_data_output/point_clouds"
GT_DIR      = "nuscenes_data_output/GT"
OUT_DIR     = "output_smoke"
MIN_POINTS  = 100    # discard tiny clouds
NUM_SAMPLES = 3      # how many nearest tokens to test
os.makedirs(OUT_DIR, exist_ok=True)

# ─── 1) collect tokens present in both dirs & above point threshold
pc_tokens = {f[:-4] for f in os.listdir(PARTIAL_DIR) if f.endswith(".ply")}
gt_tokens = {f[:-4] for f in os.listdir(GT_DIR)      if f.endswith(".ply")}
common    = pc_tokens & gt_tokens

good = []
for tok in common:
    pcd = o3d.io.read_point_cloud(f"{PARTIAL_DIR}/{tok}.ply")
    if len(pcd.points) >= MIN_POINTS:
        centroid = np.asarray(pcd.get_center())
        dist     = np.linalg.norm(centroid)  # distance to origin
        good.append((tok, dist))

if not good:
    raise RuntimeError("No token passed the MIN_POINTS filter")

# ─── 2) pick the ones with smallest centroid distance
good.sort(key=lambda x: x[1])
sample = [tok for tok, _ in good[:NUM_SAMPLES]]
print(f"Testing these {len(sample)} closest tokens:", sample)

# ─── 3) run the pipeline on those with minimal settings
for tok in sample:
    cd = run_completion(
        token           = tok,
        output_dir      = OUT_DIR,
        gen_rgb         = False,  # skip heavy ControlNet→RGB
        inpaint_depth   = False,  # skip hole-filling
        inference_steps = 10,     # fewer diffusion steps
        seed            = 0
    )
    print(f"{tok} → Chamfer distance {cd:.3f} mm")


In [ ]:
# 1. Uninstall conflicting HF hub & diffusers
!pip uninstall -y huggingface_hub diffusers



In [ ]:

# 2. Re-install matching versions
!pip install -q \
    huggingface_hub==0.13.4 \
    diffusers==0.33.1

In [ ]:
!pip uninstall -y torch torchvision torchaudio diffusers transformers


In [ ]:
!pip install --no-cache-dir \
    torch==2.2.1+cu118 \
    torchvision==0.17.1+cu118 \
    torchaudio==2.2.1+cu118 \
    -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install \
    diffusers==0.20.1 \
    transformers==4.33.1 \
    xformers \
    open3d \
    rembg \
    hy3dgen

In [ ]:
import torch, torchvision
from torchvision.ops import nms
import diffusers, transformers
print(torch.__version__, torchvision.__version__,
      transformers.__version__, diffusers.__version__)


In [ ]:
!pip uninstall -y torchao


In [ ]:
!pip install --no-cache-dir torchaudio==2.2.1+cu118 \
  -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!pip uninstall -y torch torchvision torchaudio


In [ ]:
!pip install --no-cache-dir \
  torch==2.2.1+cu118 \
  torchvision==0.17.1+cu118 \
  torchaudio==2.2.1+cu118 \
  -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torch, torchvision
from torchvision.ops import nms

print(torch.__version__, torchvision.__version__)
print("NMS op loaded:", nms)


In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# pick your token
token = "25bfb10d641f42d78edd97c1f9849a66"
pcd_path = f"nuscenes_data_output/point_clouds/{token}.ply"

# read & down-sample for speed
pcd = o3d.io.read_point_cloud(pcd_path)
pcd = pcd.voxel_down_sample(voxel_size=0.01)

# if your runtime can pop up an Open3D window:
o3d.visualization.draw_geometries([pcd])

# otherwise fall back to matplotlib:
pts = np.asarray(pcd.points)
fig = plt.figure(figsize=(6,6))
ax  = fig.add_subplot(111, projection='3d')
ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=1, c=pts[:,2], cmap='Spectral')
ax.set_title(f"Original partial cloud for token {token}")
plt.show()


In [ ]:
!ls -R .


In [ ]:
!ls /content/drive/MyDrive/COS526-Project-main/output


In [ ]:
!pip uninstall -y torch torchvision torchaudio xformers

In [ ]:
!pip install --no-cache-dir \
    torch torchvision torchaudio

In [ ]:
!pip install --no-cache-dir \
    open3d diffusers rembg hy3dgen transformers accelerate

In [ ]:
from shape_completion_pipelinenuscenes import run_completion
import os, random
import open3d as o3d

PARTIAL_DIR = "nuscenes_data_output/point_clouds"
GT_DIR      = "nuscenes_data_output/GT"
MIN_POINTS  = 100

# filter tokens
common = set(os.listdir(PARTIAL_DIR)) & set(os.listdir(GT_DIR))
good   = [t[:-4] for t in common if t.endswith(".ply")
          and len(o3d.io.read_point_cloud(f"{PARTIAL_DIR}/{t}").points)>=MIN_POINTS]

sample = random.sample(good, min(2, len(good)))
os.makedirs("output", exist_ok=True)

# run on a few examples
for tok in sample:
    cd = run_completion(
        token           = tok,
        output_dir      = "output",
        gen_rgb         = True,
        inpaint_depth   = True,
        inference_steps = 30,
        seed            = 0
    )
    print(f"{tok} → Chamfer {cd:.3f} mm")

In [ ]:
# ─── lightweight smoke-test wrapper for Colab ─────────────────────────────
import os, random
import open3d as o3d
from shape_completion_pipelinenuscenes import run_completion

# 1) paths & thresholds
PARTIAL_DIR = "nuscenes_data_output/point_clouds"
GT_DIR      = "nuscenes_data_output/GT"
MIN_POINTS  = 100
NUM_SAMPLES = 2     # ← keep this small!
OUT_DIR     = "output_smoke"
os.makedirs(OUT_DIR, exist_ok=True)

# 2) gather tokens present in both dirs and ≥MIN_POINTS
pc    = {f[:-4] for f in os.listdir(PARTIAL_DIR) if f.endswith(".ply")}
gt    = {f[:-4] for f in os.listdir(GT_DIR)      if f.endswith(".ply")}
common = pc & gt
good   = [tok for tok in common
          if len(o3d.io.read_point_cloud(f"{PARTIAL_DIR}/{tok}.ply").points) >= MIN_POINTS
          and len(o3d.io.read_point_cloud(f"{GT_DIR}/{tok}.ply").points)     >= MIN_POINTS]

print(f"Found {len(good)} good tokens, sampling {NUM_SAMPLES}…")
sample = random.sample(good, min(NUM_SAMPLES, len(good)))

# 3) run with minimal settings
for tok in sample:
    cd = run_completion(
        token           = tok,
        output_dir      = OUT_DIR,
        gen_rgb         = False,   # skip heavy ControlNet→RGB
        inpaint_depth   = False,   # skip hole-fill
        inference_steps = 10,      # fewer diffusion steps
        seed            = 0
    )
    print(f"{tok} → Chamfer {cd:.3f} mm")


In [ ]:
"widgets" : {
    "application/vnd.jupyter.widget-state+json": {
        "state": {},
    }
}